In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import re
import os
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import joblib

import nltk

nltk.data.path.append('/Users/akashmukherjee/nltk_data')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


False

In [2]:
df=pd.read_csv('../data/final_hateXplain.csv')
df

,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation,NaN
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation,NaN
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive,African,Nonreligious,No_gender,No_orientation,NaN
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech,Arab,Islam,No_gender,No_orientation,NaN
20106,🤣 this so ghetto i ’ m ready to get tf,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech,Arab,Nonreligious,No_gender,No_orientation,NaN


In [ ]:
df.info()

In [ ]:
df['Miscellaneous'].fillna('None', inplace=True)
df

In [ ]:
df['Features']=df['Race']+' '+df['Religion']+' '+df['Gender']+' '+df['Sexual Orientation']+' '+df['Miscellaneous']
df['text']=df['comment']+' '+df['Features']
df

In [ ]:
def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    
    # Remove hashtags and mentions
    tokens = [re.sub(r'#\S+', '', token) for token in tokens]
    tokens = [re.sub(r'@\S+', '', token) for token in tokens]
    
    # Remove emojis
    tokens = [token for token in tokens if not emoji.is_emoji(token)]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join tokens back into a single string
    text = ' '.join(tokens)
    
    return text

In [ ]:
df['text']=df['text'].apply(preprocess)

In [ ]:
X = df['text']
y = df['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_val_tfidf=tfidf.transform(X_val)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [ ]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf,y_train_encoded)

In [ ]:
predictions=model.predict(X_val_tfidf)
accuracy_score(y_true=y_val_encoded,y_pred=predictions)

In [ ]:
model=RandomForestClassifier(n_estimators=400)
model.fit(X_train_tfidf,y_train_encoded)

In [ ]:
predictions=model.predict(X_val_tfidf)
accuracy_score(y_true=y_val_encoded,y_pred=predictions)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

In [ ]:
# Set up GridSearchCV
rs_cv = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                           cv=5, n_jobs=-1,  scoring='accuracy')

In [ ]:
rs_cv.fit(X_train_tfidf,y_train_encoded)

In [ ]:
best_params=rs_cv.best_params_
rs_cv.best_score_

In [ ]:
best_params

In [ ]:
os.makedirs('../models',exist_ok=True)

In [ ]:
joblib.dump(model,'../models/model.joblib')

In [ ]:
joblib.dump(label_encoder,'../models/encoder.joblib')